#                                        modèle de prédiction de moyenne future 

# Étape 1 : Connexion et Chargement des Données :

In [3]:
import pandas as pd
import sqlalchemy

# Connexion à la base SQL Server
engine = sqlalchemy.create_engine("mssql+pyodbc://DESKTOP-NTA9A3K/DW_PI7?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Chargement des données nécessaires
query = """
    SELECT 
        f.score_final,
        f.moy_bac_et,
        d.Mention,
        b.nature_bac
    FROM fact_date2 f
    JOIN Dim_Diplome d ON f.Diplome_Pk = d.Diplome_Pk
    JOIN Dim_Bac b ON f.Bac_Pk = b.Bac_Pk
    WHERE f.score_final IS NOT NULL
"""
df = pd.read_sql(query, engine)

# Étape 2 : Nettoyage et Préparation des Données :

In [6]:
# Nettoyage des mentions (unification des formats)
def clean_mention(mention):
    mention = str(mention).strip()
    if mention in ['جيد جداً', 'جيد جدا ']:
        return 'جيد جدا'
    return mention

df['Mention'] = df['Mention'].apply(clean_mention)

# Vérification des valeurs uniques
print("Mentions uniques:", df['Mention'].unique())

Mentions uniques: ['جيد' 'مقبول' 'جيد جدا' 'ممتاز']


# Étape 3 : Création de la Variable Cible :

In [9]:
# Création d'une moyenne future synthétique (échelle 0-20)
df['moyenne_future'] = (df['score_final']/5 * 0.6) + (df['moy_bac_et'] * 0.4)

# Ajout d'un léger bruit aléatoire pour variabilité
import numpy as np
df['moyenne_future'] += np.random.normal(0, 0.5, len(df))

# Bornage entre 8 et 20 (plage réaliste)
df['moyenne_future'] = np.clip(df['moyenne_future'], 8, 20)

# Étape 4 : Encodage des Variables Catégorielles

In [12]:
# Ordre hiérarchique des mentions
mention_order = ['مقبول', 'جيد', 'جيد جدا', 'ممتاز']

# Encodage numérique des mentions
df['Mention_encoded'] = df['Mention'].map({m:i for i,m in enumerate(mention_order)})

# Vérification des valeurs manquantes
print("Valeurs manquantes:", df['Mention_encoded'].isna().sum())

Valeurs manquantes: 0


In [14]:
# Ordre hiérarchique des mentions
mention_order = ['مقبول', 'جيد', 'جيد جدا', 'ممتاز']

# Encodage numérique des mentions
df['Mention_encoded'] = df['Mention'].map({m:i for i,m in enumerate(mention_order)})

# Vérification des valeurs manquantes
print("Valeurs manquantes:", df['Mention_encoded'].isna().sum())

Valeurs manquantes: 0


# Étape 5 : Préprocessing des Données :

In [17]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Définition des transformations
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), ['score_final', 'moy_bac_et']),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['nature_bac']),
    ('mention', 'passthrough', ['Mention_encoded'])
])

# Étape 6 : Construction du Modèle :

In [20]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Définition des transformations
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), ['score_final', 'moy_bac_et']),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['nature_bac']),
    ('mention', 'passthrough', ['Mention_encoded'])
])

# Étape 7 : Entraînement du Modèle :

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# Création du pipeline
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=300,
        max_depth=7,
        min_samples_leaf=5,
        random_state=42
    ))
])

# Étape 8 : Évaluation du Modèle :

In [26]:
from sklearn.model_selection import train_test_split

# Séparation des données
X = df[['score_final', 'moy_bac_et', 'nature_bac', 'Mention_encoded']]
y = df['moyenne_future']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['score_final',
                                                   'moy_bac_et']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['nature_bac']),
                                                 ('mention', 'passthrough',
                                                  ['Mention_encoded'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=7, min_samples_leaf=5,
                                       n_estimators=300, random_state=42))])

In [28]:
from sklearn.model_selection import train_test_split

# Séparation des données
X = df[['score_final', 'moy_bac_et', 'nature_bac', 'Mention_encoded']]
y = df['moyenne_future']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['score_final',
                                                   'moy_bac_et']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['nature_bac']),
                                                 ('mention', 'passthrough',
                                                  ['Mention_encoded'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=7, min_samples_leaf=5,
                                       n_estimators=300, random_state=42))])

# Étape 8 : Évaluation du Modèle :

In [31]:
from sklearn.metrics import r2_score, mean_squared_error

# Prédiction sur le jeu de test
y_pred = model.predict(X_test)

# Calcul des métriques
print("R²:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))

R²: 0.9047019720806312
RMSE: 0.9859877626986223


C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Étape 9 : Sauvegarde du Modèle :

In [34]:
import joblib

# Sauvegarde pour utilisation future
joblib.dump(model, 'modele_moyenne_future.pkl')

['modele_moyenne_future.pkl']

# Étape 10 : Fonction de Prédiction :

In [37]:
def predict_moyenne(score, moy_bac, mention, nature_bac):
    """Prédit la moyenne future sur 20"""
    # Nettoyage de l'entrée
    mention = clean_mention(mention)
    
    # Création du DataFrame d'entrée
    input_data = pd.DataFrame([{
        'score_final': score,
        'moy_bac_et': moy_bac,
        'nature_bac': nature_bac,
        'Mention_encoded': mention_order.index(mention)
    }])
    
    # Prédiction et bornage
    prediction = model.predict(input_data)[0]
    return max(8, min(20, prediction))

# Exemple d'utilisation
print(predict_moyenne(85, 14, 'جيد جدا', 'ECOG'))

15.54349463544625


# Étape 11 : Vérification Finale :

In [40]:
# Test sur plusieurs cas
test_cases = [
    (85, 14, 'جيد جدا', 'ECOG'),
    (70, 12, 'جيد', 'MATH'),
    (95, 16, 'ممتاز', 'SCEXP')
]

for score, moy, mention, bac in test_cases:
    pred = predict_moyenne(score, moy, mention, bac)
    print(f"Input: {score}, {moy}, {mention}, {bac} -> {pred:.2f}/20")

Input: 85, 14, جيد جدا, ECOG -> 15.54/20
Input: 70, 12, جيد, MATH -> 14.08/20
Input: 95, 16, ممتاز, SCEXP -> 16.53/20
